In [ ]:
from sklearn.cluster import KMeans

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

***Saved Without Images to Prevent Overloading GitHub***

In [ ]:
n_samps_cluster = 1000
x_mns =[-2.7,-2.7,-2.7,-1.5,0]
y_mns =[1.2 ,1.8 ,2.8 ,2.3 ,2.5]
var = [.1,.1,.1,.3,.25]

X = [[],[]]
[X[0].extend(np.random.normal(loc = x_mns[k],scale = var[k],size = n_samps_cluster)) for k in range(5)];
[X[1].extend(np.random.normal(loc = y_mns[k],scale = var[k],size = n_samps_cluster)) for k in range(5)];
X = np.array(X).T


def plot_raw_data(X):
    plt.scatter(X[:,0],X[:,1],color='red')
    plt.ylim([.8,3.5])
    plt.xlim([-3,1.2])
    
plot_raw_data(X)

## K Means

In [ ]:
k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_transform(X)

Centroids of the clusters

In [ ]:
kmeans.cluster_centers_

In [ ]:
def plot_boundaries(x_range,y_range,model):
    xgrid,ygrid = np.meshgrid(x_range,y_range)
    mesh_grid_cols = model.predict(np.c_[xgrid.flatten(),ygrid.flatten()])
    plt.contourf(x_range,y_range,mesh_grid_cols.reshape(xgrid.shape[0],-1))

In [ ]:
plot_boundaries(np.arange(-3,1.2,1/100),np.arange(.8,3.5,1/100),kmeans)
plot_raw_data(X)

Add new predictions 

In [ ]:
xnew = np.array([[-2,-1.5,-.5,.5],[1.0,1.5,3.2,1.0]]).T
ynew = kmeans.predict(xnew)
print(ynew)
print(kmeans.transform(xnew))



In [ ]:
print(kmeans.inertia_)
print(kmeans.score(X))

Mini Batch K Means 

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import time

inertia_km, inertia_mb, time_km, time_mb = [],[],[],[]

all_k = np.arange(2,40,2)
for k in all_k:
    start_mb = time.time()
    mb = MiniBatchKMeans(n_clusters=k,random_state=42).fit(X)
    time_mb.append(time.time() - start_mb)
    
    inertia_mb.append(mb.inertia_)
    
    start_km = time.time()
    km = KMeans(n_clusters=k,random_state = 42).fit(X)
    time_km.append(time.time() - start_km)
    
    inertia_km.append(km.inertia_)
    
    
plt.subplot(121);
plt.plot(all_k,inertia_mb,color='blue',label='Mini-batch K-Means')
plt.plot(all_k,inertia_km,color='red',label='K-Means')
plt.title('Inertia')
plt.xlabel('K')
plt.legend()

plt.subplot(122);
plt.plot(all_k,time_mb,'-ob')
plt.plot(all_k,time_km,'--r')
plt.title('Training time (seconds)')

## Silhouttes

In [ ]:
from sklearn.metrics import silhouette_score, silhouette_samples

inertia_km = []
silhoutte  = []
sub_x, sub_y = [],[]

n_samps = X.shape[0]
all_k = np.arange(2,9)
for k in all_k:
    km = KMeans(n_clusters=k,random_state = 42).fit(X)
    
    inertia_km.append(km.inertia_)
    silhoutte.append(silhouette_score(X,km.labels_))
    
    if k in [3,4,5,6]:
        this_x,this_y = [],[]
        
        sil_samps = silhouette_samples(X,km.labels_)
        for c in range(k):
            ind_this_k = np.argwhere(km.labels_ == c)

            this_x.append(np.sort(sil_samps[ind_this_k.flatten()]))
            this_y.append(np.linspace(c - len(ind_this_k) / n_samps,c + len(ind_this_k) / n_samps,len(ind_this_k)))
                        
        sub_x.append(this_x)
        sub_y.append(this_y)

plt.figure(figsize=(10,4))

plt.subplot(121)
plt.plot(all_k,inertia_km,'-ob')
plt.xticks(range(1,8),labels=range(1,8));
plt.ylabel('Inertia')

plt.subplot(122)
plt.plot(all_k,silhoutte,'-ob')
plt.xticks(range(1,8),labels=range(1,8));
plt.ylabel('Silhoutte score')

In [ ]:
figgy = plt.figure(figsize=(10,10))
my_colors = get_cmap('tab10').colors
for i,k in enumerate([3,4,5,6]):
    for c in range(len(sub_x[i])):
        plt.subplot(2,2,i+1)
        
        plt.plot(sub_x[i][c],sub_y[i][c],color=my_colors[c])
        plt.fill_betweenx(sub_y[i][c],sub_x[i][c],np.min(sub_x[i][c]),color=my_colors[c])
        
        plt.title(f'k = {k}')
        plt.ylabel('Cluster')
        plt.xlabel('Silhouette Coefficient')

Clustering for PreProcessing

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

X_dig,y_dig = load_digits(return_X_y=True)
X_train,X_test,y_train,y_test = train_test_split(X_dig,y_dig,test_size=.2)

log_reg = LogisticRegression(max_iter = 5000).fit(X_train,y_train)
log_reg.score(X_test,y_test)


In [ ]:
from sklearn.pipeline import Pipeline

full_pipeline = Pipeline([
    ('k_means',KMeans(n_clusters = 50)),
    ('log_reg',LogisticRegression(max_iter = 10000))
])

full_pipeline.fit(X_train,y_train)
full_pipeline.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'k_means__n_clusters': np.arange(96,101)}

grid_cv = GridSearchCV(full_pipeline,params,cv = 3,verbose=2).fit(X_train,y_train)
print(grid_cv.best_params_)
grid_cv.score(X_test,y_test)


## Semi-supervised training. 

In [ ]:
n_labeled = 50
log_reg = LogisticRegression(max_iter = 5000).fit(X_train[:n_labeled],y_train[:n_labeled])
log_reg.score(X_test,y_test)

In [ ]:
y_train[rep_indx]

In [ ]:
k = 50
kmeans = KMeans(n_clusters = k,random_state=42)
X_digits = kmeans.fit_transform(X_train)
rep_indx = np.argmin(X_digits,axis=0)
rep_digits = X_train[rep_indx]

figgy = plt.figure(figsize=(8,8))
for i in range(len(rep_digits)):
    plt.subplot(5,10,i+1)
    plt.imshow(rep_digits[i].reshape(8,8))
    plt.xticks([]); plt.yticks([])

rep_labels = [5,8,1,0,1,7,9,2,6,3,
              4,5,3,7,4,4,5,2,8,2,
              2,6,0,1,1,3,8,6,6,9,
              4,0,7,9,7,3,5,8,9,2,
              9,1,7,5,8,7,4,3,1,4]

Manually label selected samples close to cluster centroids

In [ ]:
log_reg = LogisticRegression(max_iter = 5000).fit(rep_digits,rep_labels)
log_reg.score(X_test,y_test)

Propagate label to all labels in the cluster

In [ ]:
y_train_propagated = np.empty(len(X_train),dtype = np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = rep_labels[i]
    
    
log_reg = LogisticRegression(max_iter =5000).fit(X_train,y_train_propagated)
log_reg.score(X_test,y_test)

Propagate label to only a small percentage of the cluster

In [ ]:
perc = 20

ind_to_train = []
for i in range(k):
    cluster_dist = X_digits[kmeans.labels_ == i , i] #find the digits for this cluster
    cut_off = np.percentile(cluster_dist,perc)  
     
    ind_to_train.extend(np.argwhere(X_digits[:, i ] < cut_off).flatten()) #identify the indices for the samples closest to the centroids
    
X_partial = X_train[ind_to_train]   
y_partial = y_train_propagated[ind_to_train]

log_reg = LogisticRegression(max_iter=5000).fit(X_partial,y_partial)
log_reg.score(X_test,y_test)

## DBScan

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X,y = make_moons(n_samples = 1000,noise = 0.05)
dbscan = DBSCAN(eps = 0.2,min_samples=5).fit(X)

dbscan.labels_[0:20]

In [ ]:
print(dbscan.core_sample_indices_[0:10])
print(dbscan.components_[0:10])


Use KNN to classify new samples to existing clusters via core samples

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50).fit(dbscan.components_,dbscan.labels_[dbscan.core_sample_indices_])

Xnew = np.array([[-0.5,0],[0,0.5],[1,-0.1],[2,1]])
print(knn.predict(Xnew))
print(knn.predict_proba(Xnew))

Anomaly detector. If distance to nearest neighbor is greater than a threshold, set to -1 as an anomally.

In [ ]:
y_dist , y_pred_idx = knn.kneighbors(Xnew,n_neighbors=1)
y_preds = dbscan.labels_[y_pred_idx]

y_preds[y_dist > 0.2] = -1 
y_preds

In [ ]:
X = [[],[]]
[X[0].extend(np.random.normal(loc = x_mns[k],scale = var[k],size = n_samps_cluster)) for k in range(5)];
[X[1].extend(np.random.normal(loc = y_mns[k],scale = var[k],size = n_samps_cluster)) for k in range(5)];
X = np.array(X).T

## Gaussian Mixtures

In [ ]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components = 3, n_init = 10).fit(X)
print(gm.weights_)
print(gm.means_)
print(gm.covariances_)

In [ ]:
print(gm.converged_)
print(gm.n_iter_)

In [ ]:
print(gm.predict(X))
print(gm.predict_proba(X))

Generative models allow for sampling

In [ ]:
Xnew,ynew = gm.sample(5)
print(Xnew)
print(ynew)

In [ ]:
print(gm.score_samples(Xnew))

In [ ]:
np.radians(30)

Creation of raw data

In [ ]:
xm = [-3,0,6]
ym = [2,0,2]
xv = [.5,.5,1]
yv = [2, 2,3]

rot_mat = lambda x,y : [x*np.cos(np.radians(-30)) - y*np.sin(np.radians(-30)) , x*np.sin(np.radians(-30)) + y*np.cos(np.radians(-30))]
xdat,ydat = [],[]

xdat.extend(np.random.normal(xm[k],xv[k],100) for k in range(3))
ydat.extend(np.random.normal(ym[k],yv[k],100) for k in range(3))

xdat[0],ydat[0] = rot_mat(xdat[0],ydat[0])
xdat[1],ydat[1] = rot_mat(xdat[1],ydat[1])

X = np.c_[np.r_[xdat[0], xdat[1],xdat[2]] , np.r_[ydat[0],ydat[1],ydat[2]]]


plt.scatter(xdat,ydat)


Fitting GM

In [ ]:
gm = GaussianMixture(n_components = 3,n_init = 10).fit(X)

plot_boundaries(np.arange(-5,10,1/100),np.arange(-10,13,1/100),gm)
plot_raw_data(X)

plt.ylim([-10,12])
plt.xlim([-5,10])


Identification of Outliers

In [ ]:
densities = gm.score_samples(X)
density_threshold = np.percentile(densities,4)
anomalies = X[densities < density_threshold]

plot_boundaries(np.arange(-5,10,1/100),np.arange(-10,13,1/100),gm)
plot_raw_data(X)
plt.scatter(anomalies[:,0],anomalies[:,1],marker= 'x',color='blue')

plt.ylim([-10,12])
plt.xlim([-5,10])

## Bayesian Gaussian Mixture Model

In [ ]:
from sklearn.mixture import BayesianGaussianMixture

bgm = BayesianGaussianMixture(n_components=10,n_init=10)
bgm.fit(X)
np.round(bgm.weights_,2)

## Exercise 10

In [ ]:
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.cluster import KMeans

X, y = fetch_olivetti_faces(random_state = 42, return_X_y= True)

In [ ]:
shuffler = StratifiedShuffleSplit(n_splits = 1,test_size = .2)

train_indx, test_indx = next(shuffler.split(X,y))
X_train,y_train = X[train_indx],y[train_indx]
X_tmp,y_tmp = X[test_indx],y[test_indx]

val_shuffler = StratifiedShuffleSplit(n_splits = 1, test_size =.5)

test_indx, val_indx = next(val_shuffler.split(X_tmp,y_tmp))
X_val,y_val = X_tmp[val_indx],y_tmp[val_indx]
X_test,y_test = X_tmp[test_indx],y_tmp[test_indx]

In [ ]:
k_vals = np.arange(2,200,5)
sil_vals,inertia_vals = [],[]
k_models = []

    
for k in k_vals:
    kmeans = KMeans(n_clusters = k).fit(X_train)
    
    inertia_vals.append(kmeans.inertia_)
    sil_vals.append(silhouette_score(X_train,kmeans.labels_))
    k_models.append(kmeans)

In [ ]:
max_sil = np.argmax(sil_vals)
best_model = k_models[max_sil]

plt.figure(figsize=(30,5))

plt.subplot(121); plt.plot(k_vals,sil_vals,'-ob'); plt.plot(k_vals[np.argmax(sil_vals)],np.max(sil_vals),color = 'r',marker='o')
plt.xlabel('K'); plt.ylabel('Silhouette value'); plt.title(f'Optimum clusters: {k_vals[np.argmax(sil_vals)]}')
plt.xticks(range(0,200,10),range(0,200,10));

plt.subplot(122); plt.plot(k_vals,inertia_vals,'-ob'); plt.plot(k_vals[np.argmax(sil_vals)],inertia_vals[np.argmax(sil_vals)],color = 'r',marker='o')
plt.xlabel('K'); plt.ylabel('Inertia value'); plt.title(f'Index: {np.argmax(sil_vals)}')
plt.xticks(range(0,200,10),range(0,200,10));


In [ ]:
X_transform = best_model.transform(X_train)
best_transforms = np.argmin(X_transform,axis=0) #find the best transforms (closest to cluster centers)

In [ ]:
counts = plt.hist(best_model.labels_,bins=50);
plt.title('Distribution of faces per cluster');


In [ ]:
nrows = best_model.n_clusters
ncols = int(counts[0].max())
fig = plt.figure(figsize = (ncols,nrows))

for c in range(best_model.n_clusters):
    plt.subplot(nrows,ncols,c*ncols+1)
    
    plt.imshow(X_train[best_transforms[c],:].reshape(64,64),cmap= 'gray'); plt.xticks([]); plt.yticks([])
    plt.ylabel(f'Cluster {c}'); plt.title(y_train[best_transforms[c]][0])
    
    other_in_cluster = np.argwhere(best_model.labels_ == c)
    for i,others in enumerate(other_in_cluster,start = 1):
        plt.subplot(nrows,ncols,c*ncols + i + 1)
        plt.imshow(X_train[others,:].reshape(64,64),cmap='gray'); plt.xticks([]); plt.yticks([]); plt.title(y_train[others[0]])


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score

import numpy as np

rfg = RandomForestClassifier(n_estimators = 120,random_state=42).fit(X_train,y_train)
ypred = rfg.predict(X_val)
r2_score(y_val,ypred)


## Exercise 11

In [ ]:
X_train_reduced = best_model.transform(X_train)
X_val_reduced = best_model.transform(X_val)

rfg = RandomForestClassifier(n_estimators = 120, random_state = 42).fit(X_train_reduced,y_train)
ypred = rfg.predict(X_val_reduced)
r2_score(y_val,ypred)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

my_pipe = Pipeline([
    ('kmean',KMeans()),
    ('rfg',RandomForestClassifier(n_estimators = 120,random_state=42))
])

params = {'kmean__n_clusters':np.arange(50,200,10)}

gs_cv = GridSearchCV(my_pipe,params,cv=3,verbose = 4, error_score = 'raise').fit(np.r_[X_train, X_val],np.r_[y_train, y_val])


In [ ]:
print(gs_cv.best_params_)
print(gs_cv.best_score_)
gs_cv.best_estimator_

In [ ]:
appended_X_train = np.c_[X_train, X_train_reduced]
appended_X_val = np.c_[X_val, X_val_reduced]

rnf = RandomForestClassifier(n_estimators = 120 , random_state=42)

rnf.fit(appended_X_train,y_train)
rnf.score(appended_X_val,y_val)

In [ ]:
print(gs_cv.best_estimator_.score(X_test,y_test))

## Exercise 12

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

my_pca = PCA(.99)


X_train_pca = my_pca.fit_transform(X_train)
X_val_pca = my_pca.transform(X_val)
X_test_pca = my_pca.transform(X_test)

In [ ]:
gmm = GaussianMixture(n_components = 40, n_init = 5,random_state = 42).fit(X_train_pca)

Generate sampled faces

In [ ]:
n_gen_faces = 20
gen_faces_reduced , y_gen_faces = gmm.sample(n_samples = n_gen_faces)
gen_faces  = my_pca.inverse_transform(gen_faces_reduced)

In [ ]:
y_gen_faces[0]

In [ ]:
plt.figure(figsize=(30,3))
for s in range(n_gen_faces):
    plt.subplot(2,20,s+1)
    plt.imshow(X_train[np.argwhere(y_train == y_gen_faces[s])[0]].reshape(64,64),cmap = 'gray'); plt.yticks([]); plt.xticks([]); plt.title(y_gen_faces[s])
    plt.subplot(2,20,20+s+1)
    plt.imshow(gen_faces[s].reshape(64,64),cmap='gray'); plt.yticks([]); plt.xticks([]); plt.title(y_gen_faces[s])
    
plt.subplot(2,20,1); plt.ylabel('First of this Category')
plt.subplot(2,20,20); plt.ylabel('Generated')

In [ ]:
rot_samps = np.random.choice(range(len(X_train)),3)
rot_good_X = X_train[rot_samps,:].reshape(-1,64,64)
rot_bad_X = np.transpose(rot_good_X,axes = (0,2,1))

flip_samps = np.random.choice(range(len(X_train)),3)
flip_good_X = X_train[flip_samps,:].reshape(-1,64,64)
flip_bad_X = flip_good_X[:,::-1]

dark_samps = np.random.choice(range(len(X_train)),3)
dark_good_X = X_train[dark_samps,:].reshape(-1,64,64)
dark_bad_X = dark_good_X * .3

all_bad = np.r_[rot_bad_X,flip_bad_X,dark_bad_X]
all_good = np.r_[rot_good_X,flip_good_X,dark_good_X]

labels = y_train[np.r_[rot_samps,flip_samps,dark_samps]]

plt.figure(figsize=(20,3))
for s in range(9):
    plt.subplot(2,9,s+1)
    plt.imshow(all_good[s],cmap = 'gray',vmin = 0, vmax = 1); plt.xticks([]); plt.yticks([]); plt.title(labels[s])
    plt.subplot(2,9,9+s+1)
    plt.imshow(all_bad[s],cmap = 'gray',vmin = 0, vmax = 1); plt.xticks([]); plt.yticks([]); plt.title(labels[s])
    
    
good_pca = my_pca.transform(all_good.reshape(9,-1))
bad_pca = my_pca.transform(all_bad.reshape(9,-1))

print(f'Average sample score for the original images: {np.mean(gmm.score_samples(good_pca))}')
print(f'Average sample score for the deformed images: {np.mean(gmm.score_samples(bad_pca))}')




## Exercise 13

In [ ]:
from sklearn.metrics import mean_squared_error

my_pca = PCA(.99)
X_train_pca = my_pca.fit_transform(X_train)
X_train_reconstructed = my_pca.inverse_transform(X_train_pca)

reconstruction_error_original = mean_squared_error(X_train,X_train_reconstructed)

X_bad = all_bad.reshape(all_bad.shape[0],-1)
X_bad_pca = my_pca.transform(X_bad)
X_bad_reconstruct = my_pca.inverse_transform(X_bad_pca)
reconstruction_error_bad = mean_squared_error(X_bad,X_bad_reconstruct)

print(f'Training reconstruction: {reconstruction_error_original}')
print(f'Morphed reconstruction: {reconstruction_error_bad}')

Plot reconstructions

In [ ]:
X_good_reconstructed = my_pca.inverse_transform(my_pca.transform(all_good.reshape(9,-1)))

plt.figure(figsize=(20,7))
for s in range(9):
    plt.subplot(4,9,s+1)
    plt.imshow(all_good[s],cmap = 'gray',vmin = 0, vmax = 1); plt.xticks([]); plt.yticks([]); plt.title(labels[s]);
    plt.subplot(4,9,9+s+1)
    plt.imshow(X_good_reconstructed[s].reshape(64,64),cmap = 'gray',vmin = 0, vmax = 1); plt.xticks([]); plt.yticks([]); plt.title(labels[s])
    
    plt.subplot(4,9,18+s+1)
    plt.imshow(all_bad[s],cmap = 'gray',vmin = 0, vmax = 1); plt.xticks([]); plt.yticks([]); plt.title(labels[s]);
    plt.subplot(4,9,27+s+1)
    plt.imshow(X_bad_reconstruct[s].reshape(64,64),cmap = 'gray',vmin = 0, vmax = 1); plt.xticks([]); plt.yticks([]); plt.title(labels[s])
    
    
plt.subplot(4,9,1); plt.ylabel('Original');
plt.subplot(4,9,10); plt.ylabel('Reconstructed Normal');
plt.subplot(4,9,19); plt.ylabel('Morphed ');
plt.subplot(4,9,28); plt.ylabel('Reconstructed Morphed');